# Data Augmentation Examples for C2A Dataset

This notebook creates visual examples of the requested augmentations applied to images in `./C2A_Dataset/new_dataset3/train/images`:

- Random Rotation (±15–30°)
- Brightness / Contrast Adjustment
- Gaussian Noise / Motion Blur
- Random Erasing / Cutout

Each augmentation is applied to a few sample images and displayed in a grid. Augmented images are also saved under `prep_outputs/` for inspection.

In [ ]:
%pip install -q albumentations opencv-python-headless matplotlib --quiet
import os
import glob
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Reproducibility for examples
RND_SEED = 42
random.seed(RND_SEED)
np.random.seed(RND_SEED)

# Paths
DATASET_TRAIN_IMAGES = './C2A_Dataset/new_dataset3/train/images'
OUT_DIR = './prep_outputs'
os.makedirs(OUT_DIR, exist_ok=True)

# Utility: load image (RGB)
def load_rgb(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Utility: show images in a row (matplotlib)
def show_row(images, titles=None, figsize=(15,5)):
    n = len(images)
    plt.figure(figsize=figsize)
    for i, img in enumerate(images):
        ax = plt.subplot(1, n, i+1)
        plt.imshow(img)
        plt.axis('off')
        if titles:
            ax.set_title(titles[i])
    plt.tight_layout()
    plt.show()

# Collect sample images (png/jpg)
all_imgs = sorted(glob.glob(os.path.join(DATASET_TRAIN_IMAGES, '*.png')) + glob.glob(os.path.join(DATASET_TRAIN_IMAGES, '*.jpg')))
if len(all_imgs) == 0:
    raise FileNotFoundError(f'No images found in {DATASET_TRAIN_IMAGES}')
sample_images = random.sample(all_imgs, min(6, len(all_imgs)))
print('Using sample images:', sample_images[:3])

In [ ]:
# Define augmentation functions/pipelines
# 1) Rotation in strict ±15–30° range (we implement explicit sampling so angle magnitude >=15)
def rotate_strict(img, min_deg=15, max_deg=30):
    # Sample sign and angle magnitude to ensure angle in [±15, ±30]
    sign = random.choice([-1, 1])
    angle = sign * random.uniform(min_deg, max_deg)
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    return rotated, angle

# 2) Brightness/Contrast adjustment (Albumentations)
bc_transform = A.RandomBrightnessContrast(brightness_limit=0.35, contrast_limit=0.35, p=1.0)

# 3) Gaussian noise OR Motion blur (choose one randomly) — use Albumentations OneOf
noise_blur = A.OneOf([
    A.GaussNoise(var_limit=(10.0, 60.0), p=1.0),
    A.MotionBlur(blur_limit=(3,7), p=1.0)
], p=1.0)

# 4) Random Erasing / Cutout (CoarseDropout)
cutout = A.CoarseDropout(max_holes=6, max_height=80, max_width=80, min_holes=1, min_height=20, min_width=20, fill_value=0, p=1.0)

# Helper to apply an albumentations transform and return RGB image
def apply_alb(transform, img):
    aug = transform(image=img)
    return aug['image']

# Test each augmentation on a single image and display
img_path = sample_images[0]
img = load_rgb(img_path)

# Rotation
rot_img, rot_angle = rotate_strict(img)
print(f'Applied rotation angle: {rot_angle:.2f} degrees')

# Brightness/Contrast
bc_img = apply_alb(bc_transform, img)

# Noise or Motion Blur
nb_img = apply_alb(noise_blur, img)

# Cutout
cut_img = apply_alb(cutout, img)

# Display row: original, rotation, brightness/contrast, noise/blur, cutout
show_row([img, rot_img, bc_img, nb_img, cut_img], titles=['Original', f'Rotation {rot_angle:.1f}°', 'Brightness/Contrast', 'Noise/MotionBlur', 'Cutout'], figsize=(20,6))

# Save these examples
base = os.path.splitext(os.path.basename(img_path))[0]
cv2.imwrite(os.path.join(OUT_DIR, base + '_orig.png'), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
cv2.imwrite(os.path.join(OUT_DIR, base + f'_rot_{int(rot_angle)}.png'), cv2.cvtColor(rot_img, cv2.COLOR_RGB2BGR))
cv2.imwrite(os.path.join(OUT_DIR, base + '_bc.png'), cv2.cvtColor(bc_img, cv2.COLOR_RGB2BGR))
cv2.imwrite(os.path.join(OUT_DIR, base + '_nb.png'), cv2.cvtColor(nb_img, cv2.COLOR_RGB2BGR))
cv2.imwrite(os.path.join(OUT_DIR, base + '_cutout.png'), cv2.cvtColor(cut_img, cv2.COLOR_RGB2BGR))
print('Saved examples to', OUT_DIR)

In [ ]:
# Create a small gallery from several sample images (original + 4 augmentations each)
n_show = min(4, len(sample_images))
for i in range(n_show):
    img_path = sample_images[i]
    img = load_rgb(img_path)
    rot_img, rot_angle = rotate_strict(img)
    bc_img = apply_alb(bc_transform, img)
    nb_img = apply_alb(noise_blur, img)
    cut_img = apply_alb(cutout, img)
    # 2x3 grid: original + 4 augs (fill last cell with info)
    fig, axes = plt.subplots(1,5, figsize=(22,6))
    imgs = [img, rot_img, bc_img, nb_img, cut_img]
    titles = ['Original', f'Rotation {rot_angle:.1f}°', 'Brightness/Contrast', 'Noise/MotionBlur', 'Cutout']
    for ax, im, t in zip(axes, imgs, titles):
        ax.imshow(im)
        ax.set_title(t)
        ax.axis('off')
    plt.tight_layout()
    display_path = os.path.join(OUT_DIR, os.path.splitext(os.path.basename(img_path))[0] + '_gallery.png')
    plt.savefig(display_path)
    plt.show()
    print('Saved gallery to', display_path)

print('Gallery generation complete. Check the', OUT_DIR, 'folder for saved images.')